In [ ]:
import numpy as np
# function to cleanup and bin categories in demographic data

def  cleanup_demographics(data):
    # remove rows with no country and birthyear information
    data = data.dropna(subset = ["COUNTRY_CODE", "BIRTH_YEAR"])

    cols = ['EXCHANGE_ACCOUNT_ID', 'level', 'CREATED_AT', 'FIRST_VERIFIED_AT',
       'STATE_CODE', 'COUNTRY_CODE', 'BIRTH_YEAR', 'OCCUPATION','SESSION_COUNT',
       'COUNT_BANKS']
    data = data[cols]
    data["FIRST_VERIFIED_AT"] = pd.to_datetime(data["FIRST_VERIFIED_AT"])
    data["CREATED_AT"] = np.where(data["CREATED_AT"]=='1/0/00',data["FIRST_VERIFIED_AT"], data["CREATED_AT"] )
    data["CREATED_AT"] = pd.to_datetime(data["CREATED_AT"])
    data["DAYS_TO_VERIFY"] = (data["CREATED_AT"]- data["FIRST_VERIFIED_AT"])
    data["DAYS_TO_VERIFY"] = data["DAYS_TO_VERIFY"].dt.days
    data["AGE"] = (datetime.datetime.now().year- data["BIRTH_YEAR"]).astype(int)
    data["COUNT_BANKS"]= data["COUNT_BANKS"].fillna(1)
    #create bins for Age
    age_levels= [19,30,40,50,60,70,80,98]
    data = category_bins(data,"AGE", age_levels)
    days_to_verify_levels = [-1,7,30,90,180,365,1148]
    data = category_bins(data, "DAYS_TO_VERIFY", days_to_verify_levels)
    # clean states
    data_states = cleanup_states(data)
    # clean countries
    data_country = clean_countries(data_states)
    # categorize similar occupation into broader buckets together
    df_occ = categorize_occupation(data_country)
    return df_occ

In [ ]:
# function to cleanup  and aggregate in Transfers data

def cleanup_transfers(data):
    data['TX_YEAR'] = pd.to_datetime(data['TX_TIME']).dt.to_period('Y')
    data["ACCOUNT_ID"] = data["ACCOUNT_ID"].astype(int)
    data["AMOUNT"] = data["AMOUNT"].astype(int)
    data = data.rename(columns = {"ACCOUNT_ID":"EXCHANGE_ACCOUNT_ID" })
    
    return data


def transfer_agg(transfers_train_encoded):  
    cols = transfers_train_encoded.columns[1:]
    transfers_agg_dict  = {c: 'max' for i, c in enumerate(cols)}
    amount = {'AMOUNT': "sum"}
    transfers_agg_dict.update(amount)
    transfers_agg_dict
    transfers_train_agg = transfers_train_encoded.groupby("EXCHANGE_ACCOUNT_ID").agg(transfers_agg_dict).reset_index()
    transfers_train_agg = transfers_train_agg.set_index("EXCHANGE_ACCOUNT_ID")
    return transfers_train_agg

In [ ]:
# function to  cleanup and aggregate Exposure data


def exposure_agg(exposure_train_encoded):
    exposure_train_agg = exposure_train_encoded.groupby("EXCHANGE_ACCOUNT_ID").agg({'category_1':'max', 'category_10':'max', 'category_11':'max',
       'category_12':'max', 'category_13':'max', 'category_14':'max', 'category_15':'max',
       'category_2':'max', 'category_3':'max', 'category_4':'max', 'category_5':'max', 'category_6':'max',
       'category_7':'max', 'category_8':'max', 'category_9':'max','SENT_INDIRECT_EXPOSURE':'sum', 'SENT_DIRECT_EXPOSURE':'sum',
       'RECEIVED_INDIRECT_EXPOSURE':'sum', 'RECEIVED_DIRECT_EXPOSURE':'sum'}).reset_index()
    exposure_train_agg = exposure_train_agg.set_index("EXCHANGE_ACCOUNT_ID")
    return exposure_train_agg

# function to clean up states - ended up dropping the colunmn as it has many categories

def cleanup_states(data):
    data['STATE_CODE'] = data['STATE_CODE'].fillna("others")
    data['STATE_CODE']= [item.strip() for item in data['STATE_CODE'].str.lower()]
    data['STATE_CODE'] = data['STATE_CODE'].map(state_dict).fillna(data['STATE_CODE'])
    final_data = convertstates(data)
    return final_data



# function to clean up states

def clean_countries(data):
    data['COUNTRY_CODE']= data['COUNTRY_CODE'].str.lower()
    #print(data['COUNTRY_CODE'].unique())
    df = convertCattoNumeric(data, "COUNTRY_CODE")
    return df
    #return data

In [ ]:
# convert categorical to numerical for each state and country
def convertstates(data):
    data_int = pd.DataFrame()

    for j in data['COUNTRY_CODE'].unique():
        l = data[(data['COUNTRY_CODE'] == j)]
        len_state = len(l['STATE_CODE'].unique())
        arr_state1 = l['STATE_CODE'].unique()

        if len_state != 1:
            arr_state2 = [i for i in range(1,len_state+1)]
        else:
            arr_state2 = [1]

        state_dict = dict(zip(arr_state1, arr_state2))        
        l['STATE_CODE_INT'] = l['STATE_CODE'].map(state_dict)
        data_int = data_int.append(l)
    return data_int


def convertCattoNumeric(df, colname):
    uni = df[colname].unique()
    col_int =[i for i in range(len(uni))]
    cat_int_dict = dict(zip(uni, col_int))
    #print(cat_int_dict)
    colname_new = colname+'_INT'
    df[colname_new] = df[colname].map(cat_int_dict).astype(int)
    return df# function to clean up occupation

def categorize_occupation(df2):
    df2['OCCUPATION'] = df2['OCCUPATION'].fillna("unknown")
    df2["OCCUPATION"] = df2['OCCUPATION'].str.lower()
    df2["OCC_CAT"] = np.where(df2["OCCUPATION"].str.contains("|".join(business)),"business",
                          np.where(df2["OCCUPATION"].str.contains("|".join(tech)),"tech",
                         np.where(df2["OCCUPATION"].str.contains("|".join(management)),"management",
                         np.where(df2["OCCUPATION"].str.contains("|".join(finance)),"finance",
                         np.where(df2["OCCUPATION"].str.contains("|".join(consultant)),"consultant",
                         np.where(df2["OCCUPATION"].str.contains("|".join(acad)),"acad",
                         np.where(df2["OCCUPATION"].str.contains("|".join(health)),"health",
                         np.where(df2["OCCUPATION"].str.contains("|".join(law)),"law",
                         np.where(df2["OCCUPATION"].str.contains("|".join(marketing)),"marketing",
                         np.where(df2["OCCUPATION"].str.contains("|".join(accounting)),"accounting",
                         np.where(df2["OCCUPATION"].str.contains("|".join(unemployed)),"unemployed",
                         np.where(df2["OCCUPATION"].str.contains("|".join(writer)),"writer",
                         np.where(df2["OCCUPATION"].str.contains("|".join(construction)),"construction",
                         np.where(df2["OCCUPATION"].str.contains("|".join(artist)),"artist",
                         np.where(df2["OCCUPATION"].str.contains("|".join(driver)),"driver",
                         np.where(df2["OCCUPATION"].str.contains("|".join(designer)),"designer",
                        np.where(df2["OCCUPATION"].str.contains("|".join(pilot)),"pilot",
                          np.where(df2["OCCUPATION"].str.contains("|".join(sports)),"sports",
                          np.where(df2["OCCUPATION"].str.contains("|".join(military)),"military",
                          np.where(df2["OCCUPATION"].str.contains("|".join(insurance)),"insurance",
                          np.where(df2["OCCUPATION"].str.contains("|".join(architect)),"architect",
                          np.where(df2["OCCUPATION"].str.contains("|".join(musician)),"musician",
                          np.where(df2["OCCUPATION"].str.contains("|".join(photographer)),"photographer",
                          np.where(df2["OCCUPATION"].str.contains("|".join(retired)),"retired",
                          np.where(df2["OCCUPATION"].str.contains("unknown"),"unknown",                   

                         "other")))))))))))))))))))))))))
    return df2

In [ ]:
# function to create bins - ended up using continous data for more variability
def category_bins(df, col, bins):
    df[col+'_GROUPS'] = pd.cut(df[col], bins=bins, labels = [i for i in range(1,len(bins))])   
    return df

    

In [ ]:
# function to create dummy variables via onehot encoding


def one_hot_ecoding(df, cols):
    feature_df = df[cols]
    onehot_df = pd.get_dummies(feature_df, columns = cols)
    demo_cat_onehot_drop = df.drop(cols, axis = 1)
    demo_cat_onehot = pd.concat([demo_cat_onehot_drop, onehot_df], axis = 1)
    return demo_cat_onehot
    